In [1]:
import sys
sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np

%jsroot on

Welcome to JupyROOT 6.30/02


In [2]:
fin = open('/data/users/mcarrigan/log/goodRunLists/currentVersion/goodRunsMerged.json')
data = json.load(fin)
x = pd.DataFrame(data['data'], columns=data['columns'])
passingFiles = x['file'].loc[(x['goodRunTight']) & (x['run'] == 1430)].to_list()


In [3]:
dataDir = '/store/user/milliqan/trees/v34/1400/'
for filename in os.listdir(dataDir):
    if not filename.endswith('.root'): continue
    if not int(filename.split('Run')[1].split('.')[0]) == 1430: continue
    if not int(filename.split('.')[1].split('_')[0]) in passingFiles: continue


In [4]:
with open('../../configuration/barConfigs/configRun1296_present.json') as config:
    pedestals = json.load(config)['pedestals']

print(pedestals)

[0.1, -0.4, 0.4, 1.2, 0.5, -0.1, 0.8, -0.4, 0.6, 0.2, 0.7, -0.2, 1.1, 0.3, 0.2, -0.2, 5.8, -0.2, 0.0, 0.6, 0.1, 0.0, 0.1, 0.2, 0.0, 0.2, -0.2, 0.0, 0.0, -0.2, 0.1, 0.0, 0.5, 0.3, 0.1, 0.0, 0.2, -0.3, -0.1, 0.2, -0.2, -0.3, 0.7, 0.1, 0.6, 0.1, 0.1, 0.0, 0.7, 1.0, 0.5, -1.9, 1.0, 1.0, -1.1, -0.1, -0.1, -0.6, 0.1, -1.2, 0.6, -0.5, 0.0, 0.1, -0.5, -0.2, 0.6, 0.1, 0.5, -0.3, -0.2, -5.3, -2.3, -2.0, 0.2, -7.8, 0.8, -0.3, 0.0, 0.2]


In [5]:
dataDir = '/store/user/milliqan/trees/v34/1400/'

filename = 'MilliQan_Run1430.1_v34.root'
fin = uproot.open(dataDir+filename)['t']
#fin = uproot.open('MilliQan_Run1430_v34.root')['t']
branches = fin.arrays()
fin.keys()

['event',
 'runNumber',
 'fileNumber',
 'boardsMatched',
 'DAQEventNumber',
 'daqFileOpen',
 'daqFileClose',
 'totalFillLumi',
 'fillId',
 'beamType',
 'beamEnergy',
 'betaStar',
 'beamOn',
 'fillStart',
 'fillEnd',
 'triggerThreshold',
 'triggerEnable',
 'triggerMajority',
 'triggerLogic',
 'dynamicPedestal',
 'sidebandMean',
 'sidebandRMS',
 'maxThreeConsec',
 'chan',
 'chanWithinBoard',
 'row',
 'column',
 'layer',
 'type',
 'board',
 'height',
 'area',
 'pickupFlag',
 'pickupFlagTight',
 'nPE',
 'riseSamples',
 'fallSamples',
 'ipulse',
 'npulses',
 'time',
 'timeFit',
 'time_module_calibrated',
 'timeFit_module_calibrated',
 'duration',
 'delay',
 'max',
 'present',
 'event_trigger_time_tag',
 'event_time',
 'event_time_fromTDC',
 'v_groupTDC_g0',
 'v_groupTDC_g1',
 'v_groupTDC_g2',
 'v_groupTDC_g3',
 'v_groupTDC_g4',
 'v_groupTDC_g5',
 'v_groupTDC_g6',
 'v_groupTDC_g7',
 'tClockCycles',
 'tTime',
 'tStartTime',
 'tTrigger',
 'tTimeDiff',
 'tMatchingTimeCut',
 'tEvtNum',
 'tRunNum

In [6]:
sbMean = branches['sidebandMean']
channels = ak.Array([np.unique(x) for x in branches['chan']])

print(sbMean[0])
print(channels[0])
print(sbMean[0][channels[0]])

nPulses = branches['npulses']
nPulses[0]

[0.699, -0.21, 0.607, -0.559, 0.164, ..., 0.564, -0.128, -0.017, 0.322, 0.127]
[13, 20, 31, 71]
[0.574, 281, 10.3, 2.24]


<Array [3, 3, 3, 6, 6, 6, 6, 6, 6, 1, 2, 2] type='12 * int32'>

In [7]:
sbMean = branches['sidebandMean']
sbMean = arr.array('d', ak.flatten(sbMean))

h_sbMean = r.TH1F('h_sbMean', "Sideband Mean", 100, -100, 100)
h_sbMean.FillN(len(sbMean), sbMean, np.ones(len(sbMean)))


In [8]:
c1 = r.TCanvas("c1", "c1", 1000, 600)
h_sbMean.Draw()
c1.SetLogy()
c1.Draw()

In [9]:
channels = branches['chan']
flatCorrection = ak.Array([pedestals[ch] for ch in ak.flatten(channels)])
flatCorrection

<Array [0.3, 0.3, 0.3, 0.1, 0.1, ..., -2, -2, 0.2, 0.2] type='46587 * float64'>

In [10]:
print(len(ak.flatten(branches['sidebandMean'])))
print(len(ak.flatten(branches['dynamicPedestal'])))
print(len(flatCorrection))
print(len(ak.flatten(branches['chan'])))

80000
80000
46587
46587


In [11]:

sbCorrected = branches['sidebandMean'] + branches['dynamicPedestal']
sbCorrected = arr.array('d', ak.flatten(sbCorrected))

h_sbMeanCorrected = r.TH1F('h_sbMeanCorrected', "Sideband Mean Corrected", 100, -100, 100)
h_sbMeanCorrected.FillN(len(sbCorrected), sbCorrected, np.ones(len(sbCorrected)))

h_sbMeanCorrected.Draw()
c1.Draw()


In [12]:
h_sbMeanCorrected.SetLineColor(2)
h_sbMean.Draw()
h_sbMeanCorrected.Draw("same")
c1.Draw()

In [13]:
h_sbMeanPulses = r.TH1F("h_sbMeanPulses", "Sideband Mean", 100, -100, 100)
h_sbMeanCorrectedPulses = r.TH1F("h_sbMeanCorrectedPulses", "Sideband Mean Corrected", 100, -100, 100)

uniqueChannels = ak.Array([np.unique(x) for x in branches['chan']])
uniqueSBMean = branches['sidebandMean'][uniqueChannels]
uniqueDynamicPedestal = branches['dynamicPedestal'][uniqueChannels]

flatCorrection = ak.Array([np.take(pedestals, ch) for ch in uniqueChannels])

uniqueSBMeanCor = uniqueSBMean + uniqueDynamicPedestal + flatCorrection

a_uniqueCor = arr.array('d', ak.flatten(uniqueSBMeanCor))
a_unique = arr.array('d', ak.flatten(uniqueSBMean))

h_sbMeanPulses.FillN(len(a_unique), a_unique, np.ones(len(a_unique)))
h_sbMeanCorrectedPulses.FillN(len(a_uniqueCor), a_uniqueCor, np.ones(len(a_uniqueCor)))

In [14]:
h_sbMeanPulses.Draw()
h_sbMeanCorrectedPulses.SetLineColor(2)
h_sbMeanCorrectedPulses.Draw("same")
c1.Draw()

In [15]:
h_sbMeanPulses_v_chan = r.TH2F("h_sbMeanPulses", "Sideband Mean", 100, -100, 100, 80, 0, 80)
h_sbMeanCorrectedPulses_v_chan = r.TH2F("h_sbMeanCorrectedPulses", "Sideband Mean Corrected", 100, -100, 100, 80, 0, 80)

a_chan = arr.array('d', ak.flatten(uniqueChannels))

h_sbMeanPulses_v_chan.FillN(len(a_chan), a_unique, a_chan, np.ones(len(a_chan)))
h_sbMeanCorrectedPulses_v_chan.FillN(len(a_chan), a_uniqueCor, a_chan, np.ones(len(a_chan)))

Warning in <TROOT::Append>: Replacing existing TH1: h_sbMeanPulses (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_sbMeanCorrectedPulses (Potential memory leak).


In [16]:
h_sbMeanPulses_v_chan.Draw('colz')
c1.Draw()
c1.SetLogy(0)

In [17]:
h_sbMeanCorrectedPulses_v_chan.Draw('colz')
c1.Draw()
c1.SetLogy(0)

In [18]:
r.gStyle.SetOptStat(0)

h_sbMeanPulses_v_chan.Draw()
h_sbMeanPulses_v_chan.GetXaxis().SetTitle("Sideband Mean")
h_sbMeanPulses_v_chan.GetYaxis().SetTitle("Channel")
h_sbMeanCorrectedPulses_v_chan.SetMarkerColor(2)
h_sbMeanCorrectedPulses_v_chan.Draw("same")
c1.Draw()
c1.SetLogy(0)
c1.Update()

l1 = r.TLegend(0.1, 0.8, 0.3, 0.9)
l1.AddEntry(h_sbMeanCorrectedPulses_v_chan, "Corrections Off", "p")
l1.AddEntry(h_sbMeanPulses_v_chan, "Default Offline", "p")
l1.Draw()


In [19]:
#dynamic + static pedestal only
uniquePedestal = uniqueDynamicPedestal + flatCorrection

a_uniquePedestal = arr.array('d', ak.flatten(uniquePedestal))

h_pedestal_v_chan = r.TH2F("h_pedestal_v_chan", "Pedestal vs Channel", 300, -200, 100, 80, 0, 80)
h_pedestal_v_chan.FillN(len(a_uniquePedestal), a_uniquePedestal, a_chan, np.ones(len(a_uniquePedestal)))


In [20]:
h_pedestal_v_chan.Draw("colz")
h_pedestal_v_chan.GetXaxis().SetTitle("Pedestal Corrections (mV)")
h_pedestal_v_chan.GetYaxis().SetTitle("Channel")
c1.Draw()

In [21]:
h_pulses_chan = r.TH2F("h_pulses_chan", "Number of Pulses Per Channel", 30, 0, 30, 80, 0, 80)
h_chan_pulses = r.TH2F("h_chan_pulses", "Number of Pulses Per Channel", 80, 0, 80, 30, 0, 30)

In [22]:
for ich in range(80):
    npChan = ak.count(branches['chan'][branches['chan']==ich], axis=1)
    mask = npChan != 0
    #npChan = npChan[mask]
    #print(ich, npChan, branches['chan'][branches['chan']==ich])
    a_npChan = arr.array('d', npChan)
    if len(a_npChan) == 0: continue
    chans = np.ones(len(a_npChan)) * ich
    a_chans = arr.array('d', chans)
    print(ich, a_npChan)
    h_pulses_chan.FillN(len(a_npChan), a_npChan, a_chans, np.ones(len(a_npChan)))
    h_chan_pulses.FillN(len(a_npChan), a_chans, a_npChan, np.ones(len(a_npChan)))

0 array('d', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [23]:
h_pulses_chan.Draw("colz")
h_pulses_chan.GetXaxis().SetTitle("Number of Pulses Per Event")
h_pulses_chan.GetYaxis().SetTitle("Channel")
c1.Draw()

In [24]:
c1 = r.TCanvas("c1", "c1", 1000, 600)

h_chan_v_pedestal = r.TH2F("h_chan_v_pedestal", "Channel vs Pedestal", 80, 0, 80, 300, -200, 100)
h_chan_v_pedestal.FillN(len(a_uniquePedestal), a_chan, a_uniquePedestal, np.ones(len(a_uniquePedestal)))

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [25]:
h_chan_v_pedestal.GetXaxis().SetTitle("Channel")
h_chan_v_pedestal.GetYaxis().SetTitle("Pedestal Shift (mV)")
h_chan_v_pedestal.Draw("")
h_chan_pulses.SetMarkerColor(2)
h_chan_pulses.Draw("same")

print(r.gPad.GetUxmax())
axis = r.TGaxis(80, 0, 80, 100, 0, 100, 510, "+L")
#axis = new TGaxis(gPad->GetUxmax(), gPad->GetUymin(), gPad->GetUxmax(), gPad->GetUymax(),
#hist3->GetMinimum(), hist3->GetMaximum(), 510, "+L");
#TGaxis *xaxis3 = new TGaxis(0, gPad->GetUymin(), gPad->GetUxmax(), gPad->GetUymin(), 0, 30, 510, "+L");

axis.SetLineColor(2)
axis.SetTitle("Number of Pulses per Event")
axis.Draw()
c1.Draw()


1.0


In [26]:
p_chan_v_pedestal = h_chan_v_pedestal.ProfileX()
p_chan_v_pedestal.GetXaxis().SetTitle("Channels")
p_chan_pulses = h_chan_pulses.ProfileX()
p_chan_pulses.SetLineColor(2)

p_ratio = p_chan_v_pedestal.ProjectionX()
p_ratio.Divide(p_chan_pulses.ProjectionX())

p_ratio.GetBinContent(2)
print(p_chan_v_pedestal.ProjectionX().GetNbinsX(), p_chan_pulses.ProjectionX().GetNbinsX())

80 80


Warning in <TROOT::Append>: Replacing existing TH1: h_chan_v_pedestal_pfx_px (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_chan_pulses_pfx_px (Potential memory leak).


In [27]:
h_proj1 = p_chan_v_pedestal.ProjectionX()
h_proj2 = p_chan_pulses.ProjectionX()
h_proj2.SetLineColor(2)
h_proj1.Draw("")
h_proj2.Draw("same")
c1.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: h_chan_v_pedestal_pfx_px (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_chan_pulses_pfx_px (Potential memory leak).


In [28]:
c1.cd()
h_ratio = r.TRatioPlot(p_chan_v_pedestal, p_chan_pulses, option='kErrorAsymmetric')
h_ratio.Draw("p")
h_ratio.GetUpperPad().cd()
p_chan_v_pedestal.Draw("p")
#p_chan_v_pedestal.GetXaxis().SetTitle("Channels")
p_chan_pulses.Draw("p same")
#p_chan_v_pedestal.GetYaxis().SetTitle("Pedestal Correction (Pulses per Event)")
p_chan_v_pedestal.GetYaxis().SetRangeUser(-16, 5)
#p_chan_v_pedestal.GetXaxis().SetTitle("Channel")
#h_ratio.GetXaxis().SetTitle("Channels")
#h_chan_v_pedestal_pfx.GetYaxis().SetTitle("PedestalCorrection (Pulses Per Event)")
#h_ratio.GetXaxis().SetTitle("Channel")
#h_ratio.GetYaxis().SetTitle("Pedestal Correction (Pulses Per Event)")
c1.Draw()
c1.SaveAs("ratioplot.root")

Warning in <TGraphAsymmErrors::Divide>: Histograms have weights: only Normal or Bayesian error calculation is supported
Info in <TGraphAsymmErrors::Divide>: Using now the Normal approximation for weighted histograms
Info in <TCanvas::SaveAs>: ROOT file ratioplot.root has been created


In [29]:
!ls ratio*

ratioplot.root


In [30]:
fout = r.TFile.Open("ratioplot.root", 'recreate')
fout.cd()
p_chan_v_pedestal.Write()
p_chan_pulses.Write()
fout.Close()